<a href="https://colab.research.google.com/github/bornbylove/libcs50/blob/master/avatarify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Avatarify Colab Server

This Colab notebook is for running Avatarify rendering server. It allows you to run Avatarify on your computer **without GPU** in this way:

1. When this notebook is executed, it starts listening for incoming requests from your computer;
1. You start the client on your computer and it connects to the notebook and starts sending requests;
1. This notebooks receives the requests from your computer, renders avatar images and sends them back;

To this end, all the heavy work is offloaded from your computer to this notebook so you don't need to have a beafy hardware on your PC anymore.


## Start the server
Run the cells below (Shift+Enter) sequentially and pay attention to the hints and instructions included in this notebook.

At the end you will get a command for running the client on your computer.

## Start the client

Make sure you have installed the latest version of Avatarify on your computer. Refer to the [README](https://github.com/alievk/avatarify#install) for the instructions.

When it's ready execute this notebook and get the command for running the client on your computer.


### Technical details

The client on your computer connects to the server via `ngrok` TCP tunnel or a reverse `ssh` tunnel.

`ngrok`, while easy to use, can induce a considerable network lag ranging from dozens of milliseconds to a second. This can lead to a poor experience.

A more stable connection could be established using a reverse `ssh` tunnel to a host with a public IP, like an AWS `t3.micro` (free) instance. This notebook provides a script for creating a tunnel, but launching an instance in a cloud is on your own (find the manual below).

# Install

### Avatarify
Follow the steps below to clone Avatarify and install the dependencies.

In [1]:
!cd /content
!rm -rf *

In [2]:
!git clone https://github.com/alievk/avatarify.git

Cloning into 'avatarify'...
remote: Enumerating objects: 1514, done.
remote: Total 1514 (delta 0), reused 0 (delta 0), pack-reused 1514 (from 1)
Receiving objects: 100% (1514/1514), 5.69 MiB | 18.72 MiB/s, done.
Resolving deltas: 100% (967/967), done.


In [3]:
cd avatarify

/content/avatarify


In [4]:
!git clone https://github.com/alievk/first-order-model.git fomm
!pip install face-alignment==1.0.0 msgpack_numpy

Cloning into 'fomm'...
remote: Enumerating objects: 211, done.
remote: Total 211 (delta 0), reused 0 (delta 0), pack-reused 211 (from 1)
Receiving objects: 100% (211/211), 58.16 MiB | 9.78 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [6]:
!pip install cython
!pip install git+https://github.com/yaml/pyyaml.git

  Cloning https://github.com/yaml/pyyaml.git to /tmp/pip-req-build-cwfgbev3
  Running command git clone --filter=blob:none --quiet https://github.com/yaml/pyyaml.git /tmp/pip-req-build-cwfgbev3
  Resolved https://github.com/yaml/pyyaml.git to commit 69c141adcf805c5ebdc9ba519927642ee5c7f639
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for PyYAML: filename=PyYAML-7.0.0.dev0-cp310-cp310-linux_x86_64.whl size=45421 sha256=5d16bc235533f1260adb1e00880f7db081b041b3a7181fec67e6b9d57f2ca025
  Stored in directory: /tmp/pip-ephem-wheel-cache-p4cxcc97/wheels/dd/37/ca/ee935e3a9d8e1e1e7007dae6ea30f8fdde8a80ab7435ad1a5a
Successfully built PyYAML
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0.2
    Uninstalling PyYAML-6.0.2:
      Successfully uninstalled PyYAML-6.0.2


In [7]:
!scripts/download_data.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  228M  100  228M    0     0   195M      0  0:00:01  0:00:01 --:--:--  195M
Expected checksum: 8a45a24037871c045fbb8a6a8aa95ebc
Found checksum:    8a45a24037871c045fbb8a6a8aa95ebc  vox-adv-cpk.pth.tar


### ngrok
Follow the steps below to setup ngrok. You will also need to sign up on the ngrok site and get your authtoken (free).


In [8]:
# Download ngrok
!scripts/get_ngrok.sh

ngrok is not found, installing...
Done!


# Run
Start here if the runtime was restarted after installation.

In [9]:
cd /content/avatarify

/content/avatarify


In [10]:
#!git pull origin

In [11]:
from subprocess import Popen, PIPE
import shlex
import json
import time


def run_with_pipe(command):
  commands = list(map(shlex.split,command.split("|")))
  ps = Popen(commands[0], stdout=PIPE, stderr=PIPE)
  for command in commands[1:]:
    ps = Popen(command, stdin=ps.stdout, stdout=PIPE, stderr=PIPE)
  return ps.stdout.readlines()


def get_tunnel_adresses():
  info = run_with_pipe("curl http://localhost:4040/api/tunnels")
  assert info

  info = json.loads(info[0])
  for tunnel in info['tunnels']:
    url = tunnel['public_url']
    port = url.split(':')[-1]
    local_port = tunnel['config']['addr'].split(':')[-1]
    print(f'{url} -> {local_port} [{tunnel["name"]}]')
    if tunnel['name'] == 'input':
      in_addr = url
    elif tunnel['name'] == 'output':
      out_addr = url
    else:
      print(f'unknown tunnel: {tunnel["name"]}')

  return in_addr, out_addr

In [12]:
# Input and output ports for communication
local_in_port = 5557
local_out_port = 5558

# Start the worker


In [13]:
# (Re)Start the worker
with open('/tmp/run.txt', 'w') as f:
  ps = Popen(
      shlex.split(f'./run.sh --is-worker --in-port {local_in_port} --out-port {local_out_port} --no-vcam --no-conda'),
      stdout=f, stderr=f)
  time.sleep(3)

This command should print lines if the worker is successfully started

In [14]:
!ps aux | grep 'python3 afy/cam_fomm.py' | grep -v grep | tee /tmp/ps_run
!if [[ $(cat /tmp/ps_run | wc -l) == "0" ]]; then echo "Worker failed to start"; cat /tmp/run.txt; else echo "Worker started"; fi

root        2923 11.1  3.0 3077656 408092 ?      S    02:29   0:02 python3 afy/cam_fomm.py --config 
root        2943  0.0  1.7 3143192 235660 ?      Sl   02:29   0:00 python3 afy/cam_fomm.py --config 
root        2944  0.0  1.7 3077656 233288 ?      S    02:29   0:00 python3 afy/cam_fomm.py --config 
root        2947  0.0  1.7 3143192 235688 ?      Sl   02:29   0:00 python3 afy/cam_fomm.py --config 
Worker started


# Open ngrok tunnel

#### Get ngrok token
Go to https://dashboard.ngrok.com/auth/your-authtoken (sign up if required), copy your authtoken and put it below.

In [15]:
# Paste your authtoken here in quotes
authtoken = "2mOSoF2TCSjXbaqka9b53WeOUjQ_6pMk8kAYozXkpB2RZLdjd"

Set your region

Code | Region
--- | ---
us | United States
eu | Europe
ap | Asia/Pacific
au | Australia
sa | South America
jp | Japan
in | India

In [16]:
# Set your region here in quotes
region = "eu"

In [17]:
config =\
f"""
version: 2
authtoken: {authtoken}
region: {region}
console_ui: False
tunnels:
  input:
    addr: {local_in_port}
    proto: tcp
  output:
    addr: {local_out_port}
    proto: tcp
"""

with open('ngrok.conf', 'w') as f:
  f.write(config)

In [28]:
# (Re)Open tunnel
ps = Popen('./scripts/open_tunnel_ngrok.sh', stdout=PIPE, stderr=PIPE)
time.sleep(3)

In [29]:
# Get tunnel addresses
try:
  in_addr, out_addr = get_tunnel_adresses()
  print("Tunnel opened")
except Exception as e:
  [print(l.decode(), end='') for l in ps.stdout.readlines()]
  print("Something went wrong, reopen the tunnel")
  # Add this line to handle the case where the tunnel fails to open
  in_addr, out_addr = None, None  # or some default values

# ... (rest of the code) ...

print('Copy-paste to the terminal the command below and run (press Enter)\n')
print('Mac:')
# Check if in_addr and out_addr are defined before using them
if in_addr and out_addr:
  print(f'./run_mac.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')
else:
  print("Tunnel addresses not available. Please ensure the tunnel is open.")
print('\nWindows:')
if in_addr and out_addr:
  print(f'run_windows.bat --is-client --in-addr {in_addr} --out-addr {out_addr}')
else:
  print("Tunnel addresses not available. Please ensure the tunnel is open.")
print('\nLinux:')
if in_addr and out_addr:
  print(f'./run.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')
else:
  print("Tunnel addresses not available. Please ensure the tunnel is open.")

Opening tunnel
start - start endpoints or tunnels by name from the configuration file

USAGE:
  ngrok start [flags]

AUTHOR:
  ngrok - <support@ngrok.com>

COMMANDS: 
  config          update or migrate ngrok's configuration file
  http            start an HTTP tunnel
  tcp             start a TCP tunnel
  tunnel          start a tunnel for use with a tunnel-group backend

EXAMPLES: 
  ngrok http 80                                                 # secure public URL for port 80 web server
  ngrok http --url baz.ngrok.dev 8080                           # port 8080 available at baz.ngrok.dev
  ngrok tcp 22                                                  # tunnel arbitrary TCP traffic to port 22
  ngrok http 80 --oauth=google --oauth-allow-email=foo@foo.com  # secure your app with oauth

Paid Features: 
  ngrok http 80 --url mydomain.com                              # run ngrok with your own custom domain
  ngrok http 80 --cidr-allow 2600:8c00::a03c:91ee:fe69:9695/32  # run ngrok with IP

### [Optional] AWS proxy
Alternatively you can create a ssh reverse tunnel to an AWS `t3.micro` instance (it's free). It has lower latency than ngrok.

1. In your AWS console go to Services -> EC2 -> Instances -> Launch Instance;
1. Choose `Ubuntu Server 18.04 LTS` AMI;
1. Choose `t3.micro` instance type and press Review and launch;
1. Confirm your key pair and press Launch instances;
1. Go to the security group of this instance and edit inbound rules. Add TCP ports 5557 and 5558 and set Source to Anywhere. Press Save rules;
1. ssh into the instance (you can find the command in the Instances if you click on the Connect button) and add this line in the end of `/etc/ssh/sshd_config`:
```
GatewayPorts yes
```
then restart `sshd`
```
sudo service sshd restart
```
1. Copy your `key_pair.pem` by dragging and dropping it into avatarify folder in this notebook;
1. Use the command below to open the tunnel;
1. Start client with a command (substitute `run_mac.sh` with `run_windows.bat` or `run.sh`)
```
./run_mac.sh --is-client --in-addr tcp://instace.compute.amazonaws.com:5557 --out-addr tcp://instance.compute.amazonaws.com:5558
```

In [ ]:
# Open reverse ssh tunnel (uncomment line below)
# !./scripts/open_tunnel_ssh.sh key_pair.pem ubuntu@instance.compute.amazonaws.com

# Start the client
When you run the cell below it will print a command. Run this command on your computer:

1. Open a terminal (in Windows open `Anaconda Prompt`);
2. Change working directory to the `avatarify` directory:</br>
* Windows (change `C:\path\to\avatarify` to your path)</br>
`cd C:\path\to\avatarify`</br></br>
* Mac/Linux (change `/path/to/avatarify` to your path)</br>
`cd /path/to/avatarify`
3. Copy-paste to the terminal the command below and run;
4. It can take some time to connect (usually up to 10 seconds). If the preview window doesn't appear in a minute or two, look for the errors above in this notebook and report in the [issues](https://github.com/alievk/avatarify/issues) or [Slack](https://join.slack.com/t/avatarify/shared_invite/zt-dyoqy8tc-~4U2ObQ6WoxuwSaWKKVOgg).

In [26]:
print('Copy-paste to the terminal the command below and run (press Enter)\n')
print('Mac:')
print(f'./run_mac.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nWindows:')
print(f'run_windows.bat --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nLinux:')
print(f'./run.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')

Copy-paste to the terminal the command below and run (press Enter)

Mac:


NameError: name 'in_addr' is not defined

# Logs

If something doesn't work as expected, please run the cells below and include the logs in your report.

In [21]:
#@title
!cat ./var/log/cam_fomm.log | head -100

[1728527376.063331] Loading Predictor


In [22]:
#@title
!cat ./var/log/recv_worker.log | tail -100

[1728527379.568475] Receiving on port 5557


In [23]:
#@title
!cat ./var/log/predictor_worker.log | tail -100

In [24]:
#@title
!cat ./var/log/send_worker.log | tail -100

[1728527379.578709] Sending on port 5558
